<a href="https://colab.research.google.com/github/HydroPanadas/CHSython/blob/main/DownloadingCHSTideData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Canadian Tide Time Series Data Downloading

**1. Set up Enviroment** italicized text

In [1]:
!pip install requests
!pip install pandas
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

**2. User Input**

The script will ask for the follow:

- Station number
- Time Series Resolution
- Start Date
- End Date
- Tide Series

In [2]:
stn_num = int(input('Please Provide Station Number: '))
data_res = input('Provide number of Resolution for Data Download 1, 3, 15, 60: ')
start_date = input('Provide Start Date in YYYY-MM-DD Format: ')
end_date = input('Provide End Date in YYYY-MM-DD Format: ')
tide_series = input('Provide Tide Series Code wlo, wlp: ').lower()

Please Provide Station Number: 07824
Provide number of Resolution for Data Download 1, 3, 15, 60: 3
Provide Start Date in YYYY-MM-DD Format: 2025-10-06
Provide End Date in YYYY-MM-DD Format: 2025-10-19
Provide Tide Series Code wlo, wlp: wlp


**3. Validate User Input**

In [3]:
valid_dates = False
while not valid_dates:
    try:
        format = '%Y-%m-%d'
        ma = datetime.strptime(end_date, format).date()
        mi = datetime.strptime(start_date, format).date()
    except ValueError:
        start_date = input(f'Provided value was {start_date}, please Provide a Start Date in YYYY-MM-DD Format: ')
        end_date = input(f'Provided value was {end_date}, please Provide a Start Date in YYYY-MM-DD Format: ')
        ma = datetime.strptime(end_date, format).date()
        mi = datetime.strptime(start_date, format).date()
        pass
    if  ma < mi:
        start_date = input(f'Start Date of {start_date} is not < End Date of {end_date}, please provide a Start Date that is less than the End Date ')
        end_date = input('Provide End Date in YYYY-MM-DD Format: ')
        valid_dates = False
    else:
        ma = datetime.strptime(end_date, format).date()
        mi = datetime.strptime(start_date, format).date()
        valid_dates = True


valid_tide_series = ['wlo', 'wlp']
valid_res = ['1', '3', '15', '60']

valid_tide = False
while not valid_tide:
     try:
          if tide_series not in valid_tide_series:
              tide_series = input(f'Provided value was {tide_series}, please Provide a valid tide series (wlo or wlp)')
          else:
              valid_tide = True
     except ValueError:
          tide_series = input(f'Provided value was {tide_series}, please Provide a valid tide series (wlo or wlp)')
          pass

valid_resolution = False
while not valid_tide:
     try:
          if data_res not in valid_res:
              data_res = input('Provide number of Resolution for Data Download 1, 3, 15, 60: ')
          else:
              valid_resolution = True
     except ValueError:
          data_res = input('Provide number of Resolution for Data Download 1, 3, 15, 60: ')
          pass

**4. Find station endpoint based on station number provided**

In [4]:
# Correcting to have Station in 5 digit format
stn_num = f"{stn_num:05d}"
stn_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={stn_num}'
stn_url

'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code=07824'

In [5]:
df = pd.read_json(stn_url)
stn_id = df['id'].to_list()
stn_id

['5dd3064ee0fdc4b9b4be670a']

**5. Program to select the correct time series resolution based on resolution input**

In [6]:
if data_res == '1':
    resolution = 'ONE_MINUTE'
elif data_res == '3':
    resolution = 'THREE_MINUTES'
elif data_res == '15':
    resolution = 'FITEEN_MINUTES'
else:
    resolution = 'SIXTY_MINUTES'

**6. Create incremental list of Min Dates until more or greater than the Max Date.**

In [7]:
format = '%Y-%m-%d'
ma = datetime.strptime(end_date, format).date()
mi = datetime.strptime(start_date, format).date()
dayoffset = 6
start_dates =[]
end_dates = []
while mi < ma:
    start_dates.append(mi.strftime(format))
    mi = mi + timedelta(days=dayoffset)
    end_dates.append(mi.strftime(format))
end_dates[-1] = end_date
print(start_dates)
print(end_dates)



['2025-10-06', '2025-10-12', '2025-10-18']
['2025-10-12', '2025-10-18', '2025-10-19']


**7. Create a list urls to request Tidal data from the API grouped by 7 day intervalsrequest until end date limit reached**

In [8]:
request_list = []

i = 0
for i in range(len(start_dates)):
  sd = start_dates[i]
  ed = end_dates[i]
  data_url = (f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/' +
                f'stations/{stn_id[0]}/data?time-series-code={tide_series}&from=' +
                f'{sd}T00%3A00%3A00Z&to={ed}T23%3A59%3A59Z&'+
                f'resolution={resolution}')
  request_list.append(data_url)


In [9]:
request_list

['https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5dd3064ee0fdc4b9b4be670a/data?time-series-code=wlp&from=2025-10-06T00%3A00%3A00Z&to=2025-10-12T23%3A59%3A59Z&resolution=THREE_MINUTES',
 'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5dd3064ee0fdc4b9b4be670a/data?time-series-code=wlp&from=2025-10-12T00%3A00%3A00Z&to=2025-10-18T23%3A59%3A59Z&resolution=THREE_MINUTES',
 'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5dd3064ee0fdc4b9b4be670a/data?time-series-code=wlp&from=2025-10-18T00%3A00%3A00Z&to=2025-10-19T23%3A59%3A59Z&resolution=THREE_MINUTES']

**8. Append data to a new Dataframe and export as a csv**

In [15]:
data = pd.DataFrame()
for url in request_list:
  df = pd.read_json(url)
  data = pd.concat([data, df])
  time.sleep(1)
data

data.to_csv(f'{stn_num}_{start_date}_{end_date}_{tide_series}_{resolution}.csv')

**9. Convert the CSV into Caris Tide File format**

In [23]:
data['date_time'] = pd.to_datetime(data['eventDate'])
data['date'] = pd.to_datetime(data['date_time']).dt.strftime("%Y/%m/%d")
data['time'] = data['date_time'].dt.time
data = data.drop(columns=['eventDate'])

tide_data = data[['date', 'time', 'value']]
fname = f'{stn_num}_{start_date}_{end_date}_{tide_series}_{resolution}'

with open(f'{fname}_CarisTide.txt', 'w') as f:
    dfAsString = tide_data.to_string(header=False, index=False)
    f.write(dfAsString)
    f.close()

with open(f'{fname}_CarisTide.tid', 'w') as f2:
    f2.write('--------\n')

    file = open(f'{fname}_CarisTide.txt', 'r')
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line=='' or line=="/n":
            pass
        else:
            f2.write(line + '\n')
    file.close()
